# Comparison to SNoW

> Given a set of web tables and a target knowledge base, the SNoW method extends each web table with additional context columns, stitches matching web tables into larger tables, and applies functional dependency discovery to identify the relations that are represented in the web tables. Further, it normalises the stitched tables, guided by the schema of the knowledge base, to create an integrated schema.

We assume that the tables are already context-enriched, and schema-unioned per Pay-Level Domain. Our task is now to match and stitch these supertables into universal tables, and decompose them into normalised relations.

In [50]:
# check: are the FK tables OK?

In [ ]:
# check: what's with the www.amoeba.com tables?

## Finding FDs

In [10]:
# Find FDs in tables, find frequent determiners
dataset_name = 'flightaware.com'
tabid_table = {t._id: t for t in takco.TableSet.dataset(benchmark_datasets[dataset_name][0])}

import pandas as pd
gold_fdeps_path = ROOT.joinpath(f'{dataset_name}/evaluation/functional_dependencies.tsv')
for row in pd.read_csv(gold_fdeps_path, sep='\t', header=None).values:
    print(row[1], '->', row[2])
print()

tane = takco.link.Tane(ROOT.parent.joinpath('tane'))
from collections import Counter
det_count = Counter()
for tabid, t in tabid_table.items():
    print(tabid)
    
    pred_fk = predict_classes(t.df, .02)
    print({t.df.columns[i]:c for i,c in pred_fk.items()})
    
    df = t.df
    df = extract_bracket_disambiguation(df)
    show = lambda cs: '[%s]'%(' '.join(map('|'.join, zip(*cs))))
    print(show(df.columns))
    
    allow_missing = 1
    main_cols = set(df.columns) - set(get_context_headers(df.columns)) - set(get_singleton_cols(df))
    for det, dep in tane.rundf(df, stoplevel=6, g3_threshold=.1).items():
        missing = main_cols - (set(det)|set(dep))
        if not missing:
            print(' ', show(set(det)), '->', show(set(dep)))
            det_count[tuple(set(det))] += 1
        elif len(missing) <= allow_missing:
            det_count[tuple(set(det))] += 1
            print(' ', show(det), '->', show(dep), 'missing:', show(missing))
    print()
    
print({show(det):c for det,c in det_count.items() if c>1})

FK,uri 2,uri 3 -> percentage of seats filled,total seats,passengers,average per flight,flights performed,flights scheduled,percentage flown
FK,page title -> cargo weight (lbs),mail transport (lbs)
FK,page title,routing,Disambiguation of carrier -> popularity,minimum,median,maximum

2.json
{}
[page title|table heading|uri 0|uri 1|uri 2|uri 3|uri 4|carrier|cargo weight (lbs)|mail transport (lbs)|disambiguation of page title]
  [cargo weight (lbs)] -> [mail transport (lbs)|carrier]
  [cargo weight (lbs)|uri 4] -> [mail transport (lbs)|carrier|uri 3|page title|uri 2]
  [cargo weight (lbs)|disambiguation of page title] -> [mail transport (lbs)|carrier|uri 4|uri 3|page title|uri 2]
  [page title|carrier] -> [mail transport (lbs)|cargo weight (lbs)]
  [uri 3|carrier|uri 2] -> [mail transport (lbs)|cargo weight (lbs)]
  [uri 2|uri 4|carrier] -> [mail transport (lbs)] missing: [cargo weight (lbs)]
  [uri 2|carrier|disambiguation of page title] -> [mail transport (lbs)] missing: [cargo weight (l

{}
[page title|table heading|uri 0|uri 1|uri 2|uri 3|uri 4|routing|carrier|popularity|minimum|median|maximum|disambiguation of page title]
  [median|uri 4] -> [carrier|minimum|popularity|maximum|routing|uri 3|page title|uri 2]
  [uri 4|maximum] -> [carrier|minimum|popularity|median|routing|uri 3|page title|uri 2]
  [disambiguation of page title|median] -> [carrier|minimum|popularity|maximum|uri 4|routing|uri 3|page title|uri 2]
  [disambiguation of page title|maximum] -> [carrier|minimum|popularity|uri 4|median|routing|uri 3|page title|uri 2]
  [median|uri 2] -> [maximum|carrier|minimum|routing|uri 3|page title|popularity]
  [uri 3|median] -> [maximum|carrier|minimum|routing|page title|uri 2|popularity]
  [page title|minimum] -> [popularity|carrier|median|routing|maximum]
  [page title|median] -> [popularity|carrier|minimum|routing|maximum]
  [carrier|median] -> [maximum|minimum|routing|uri 3|page title|uri 2|popularity]
  [minimum|popularity] -> [carrier|median|routing|uri 3|page titl

In [435]:
# %%time
# Schema matching
dataset_name = 'd3football.com'
tabid_table = {t._id: t for t in takco.TableSet.dataset(benchmark_datasets[dataset_name][0])}

tane = takco.link.Tane(ROOT.parent.joinpath('tane'))

tabid_df = {tid:extract_bracket_disambiguation(t.df, .1) for tid, t in tabid_table.items()}

import numpy as np, pandas as pd

def aggr_by_val(items):
    aggr = {}
    for k,v in items:
        aggr.setdefault(v, set()).add(k)
    return aggr

# Column similarities
from sklearn.feature_extraction.text import TfidfVectorizer
data_vectorizer = TfidfVectorizer(max_df=0.9)
tabid_and_colnr_to_colid, colid_to_tabid = {}, {}
colid_to_text = {}
for tabid, df in tabid_df.items():
    numeric_cols = set(get_numeric_cols(df))
    context_cols = set(get_context_headers(df.columns))
    singleton_cols = set(get_singleton_cols(df))
    for colnr, c in enumerate(df):
        text = ''
        if (not c in numeric_cols) and (not c in (singleton_cols & context_cols)):
            text = ' '.join(set(df[c].astype('str')))
        other_text.add(text)
        colid = f"{tabid}~Col{colnr} {c}"
        colid_to_text[colid] = text
        tabid_and_colnr_to_colid[(tabid, colnr)] = colid
        colid_to_tabid[colid] = tabid
        
D = data_vectorizer.fit_transform(colid_to_text.values())
index = pd.MultiIndex.from_tuples(colid_to_tabid.items())
colsim = pd.DataFrame(D.dot(D.T).todense(), index=index, columns=index).stack().stack()
colsim.index.names = ('ci1','ti1','ti2','ci2')
agg_threshold_col = 0.5
colsim = colsim[colsim > agg_threshold_col].reset_index()

## Cluster columns
colsim = colsim[(colsim["ti1"] != colsim["ti2"])]
colsim = colsim.set_index(["ci1", "ci2"])[0]
colsim = colsim[~colsim.index.duplicated()]

# Make symmetric distance matrix
d = 1 - colsim.unstack().sort_index(0).sort_index(1).fillna(0)
d = pd.DataFrame(np.minimum(d, d.T))

# Run clustering
from sklearn.cluster import AgglomerativeClustering
clus = AgglomerativeClustering(affinity='precomputed', linkage='complete', n_clusters=None, distance_threshold=.95)
clusters = clus.fit(d)
colid_partcolid = dict(zip(d.index, clusters.labels_))
partcolid_colids = aggr_by_val( colid_partcolid.items() )
display('partcolid_colids', partcolid_colids)

# Connected components
tabid_partid = {tabid:i for i, tabid in enumerate(tabid_table.keys())}
for colids in partcolid_colids.values():
    partid = None
    for colid in colids:
        tabid = colid_to_tabid[colid]
        if partid is None:
            partid = tabid_partid[tabid]
        tabid_partid[tabid] = partid

# Stitch tables
from collections import Counter
partid_tabids = aggr_by_val( tabid_partid.items() )
partcolid_names = {}
partid_df = {}
partid_head = {}
partid_keys = {}
partid_colnr_fk = {}
for partid, tabids in partid_tabids.items():
    aligned_tables = []
    for tabid in tabids:
        df = tabid_df[tabid]
        colname_partcolid = {}
        for colnr, colname in enumerate(df):
            colid = tabid_and_colnr_to_colid[(tabid, colnr)]
            if (colid not in colid_partcolid):
                if (colname in get_context_cols(df)):
                    continue # ignore unaligned context columns
                partcolid = colid_partcolid.setdefault(colid, len(partcolid_colids))
                partcolid_colids.setdefault(partcolid, set()).add(colid)
            partcolid = colid_partcolid[colid]
            colname_partcolid[colname] = partcolid
            partcolid_names.setdefault(partcolid, Counter()).update( colname )
        aligned = df[colname_partcolid.keys()].set_axis(colname_partcolid.values(), axis=1)
        aligned_tables.append( aligned )
    
    df = pd.concat(aligned_tables)
    head = list(zip(*(tuple(dict(partcolid_names[pci].most_common(1))) for pci in df.columns)))
    df.columns = list(range(df.shape[1]))
    partid_df[partid] = df
    partid_head[partid] = head
    
    # Predict classes
    colnr_fk = predict_classes(df.set_axis(range(len(df.columns)), axis=1))
    partid_colnr_fk[partid] = colnr_fk
    
    # Get partition keyset
    colnames = list(zip(*head))
    context_cols = set(colnames.index(c) for c in get_context_headers(colnames))
    keys = set()
    allow_missing = 1
    main_cols = set(df.columns) - set(context_cols) - set(get_singleton_cols(df))
    try:
        for det, dep in tane.rundf(df, stoplevel=6, g3_threshold=.05).items():
            missing = main_cols - (set(det)|set(dep))
            if len(missing) <= allow_missing:
                if len(set(det) - set(context_cols)) - len(missing) > len(keys):
                    if any(c in colnr_fk for c in det):
                        keys = det
    except Exception as e:
        print('Tane error:', e)
    partid_keys[partid] = keys
    
    sch = lambda cs: '[%s]'%(' '.join(map('|'.join, zip(*(colnames[c] for c in cs)))))
    print(partid, tabids)
    print(' ', sch(set(keys)), '->', sch(set(df.columns)-set(keys)))
    print(' ', colnr_fk)
print()


# Decompose FDs
def decompose_fd_tables(df, keys, header=None):
    for c in df.columns:
        if (c not in keys) and (len(set(df[c])) > 1):
            fd_df = df[[c] + list(keys)]
            
            # get filled unique rows
            filled_mask = fd_df.fillna(False).applymap(bool).any(axis=1)
            fd_df = fd_df[filled_mask].drop_duplicates(ignore_index=True)
            
            if header is not None:
                head = list(zip(*header[[c] + list(keys)]))
                _id = f"fd_{header[c][0].replace(' ','_')}"
            else:
                head = list(zip(*fd_df.columns))
                _id = f"fd_{fd_df.columns[c][0].replace(' ','_')}"
                
            yield takco.Table(head=head, body=fd_df.values, _id=_id)

def write_snow(t, name, fd_path):
    doc = takco.evaluate.dataset.WebDataCommons.convert_back(t, snow=True)
    fname = Path(fd_path).joinpath(name)
    with open(fname, 'w') as fw:
        json.dump(doc, fw, ensure_ascii=False)
            
fd_path = ROOT.joinpath(f'{dataset_name}/normalised_X_fd_relations')
!rm -r $fd_path
Path(fd_path).mkdir(parents=True,exist_ok=True)

class_value_fk = {}
from collections import Counter
class_nfds = Counter()
for partid, df in partid_df.items():
    keys = partid_keys[partid]
    columns = list(zip(*partid_head[partid]))
    colnr_fk = partid_colnr_fk[partid]
    if any(k in colnr_fk for k in keys):
        # split off top-scoring FK in keys, add its values to FK table
        fkcolnr = max(keys, key=lambda k: colnr_fk.get(k, {}).get('score', 0))
        fkclass = colnr_fk.get(fkcolnr, {}).get('class')
        value_fk = class_value_fk.setdefault(fkclass, {})
        prefix = f"{fkclass}_{dataset_name}"
        fks = [f"{prefix}~Row{value_fk.setdefault(v, len(value_fk))}" for v in df[fkcolnr]]
        df = df.fillna('')
        df[fkcolnr] = fks
        columns[fkcolnr] = ('FK',)
        
        for c in df:
            if looks_numeric(df[c]):
                df[c] = make_numeric(df[c]).fillna('').astype('str')
    
        # for each non-key column, make a table object and write it out
        for t in decompose_fd_tables(df, keys, pd.Series(columns)):
            class_nfds[fkclass] += 1
            name = f"{prefix}_fd_{class_nfds[fkclass]}.json"
            print(name)
            write_snow(t, name, fd_path)
        
    else:
        print(f"Partition {partid} has no fk in keys {keys}!")

for fkclass, value_fk in class_value_fk.items():
    value_fk = class_value_fk['VideoGame']
    body = [(f"{fkclass}_{dataset_name}~Row{i}", val) for val, i in value_fk.items()]
    head = [('PK', 'rdf-schema#label')]
    fktable = takco.Table(head=head, body=body)
    name = f"{fkclass}_{dataset_name}.json"
    print(name)
    write_snow(t, name, fd_path)
        
fd_dataset = takco.evaluate.dataset.WebDataCommons(fnames=list(fd_path.glob("*.json")))
takco.TableSet.dataset(fd_dataset).preview(ntables=None)

'partcolid_colids'

{1: {"1.json~Col0 ('page title',)",
  "2.json~Col0 ('page title',)",
  "3.json~Col0 ('page title',)",
  "4.json~Col0 ('page title',)",
  "5.json~Col0 ('page title',)",
  "6.json~Col0 ('page title',)",
  "7.json~Col0 ('page title',)",
  "8.json~Col0 ('page title',)",
  "9.json~Col0 ('page title',)"},
 2: {"1.json~Col5 ('uri 3',)",
  "2.json~Col5 ('uri 3',)",
  "3.json~Col5 ('uri 3',)",
  "4.json~Col5 ('uri 3',)",
  "5.json~Col5 ('uri 3',)",
  "6.json~Col5 ('uri 3',)",
  "7.json~Col5 ('uri 3',)",
  "8.json~Col5 ('uri 3',)",
  "9.json~Col5 ('uri 3',)"},
 0: {"1.json~Col6 ('kickoff returns',)",
  "2.json~Col6 ('passing',)",
  "3.json~Col6 ('receiving',)",
  "4.json~Col6 ('rushing',)",
  "5.json~Col6 ('interception returns',)",
  "6.json~Col6 ('punt returns',)"},
 4: {"7.json~Col6 ('punting',)"},
 3: {"9.json~Col6 ('kickoffs',)"}}

KeyboardInterrupt: 

In [436]:
# make our own Normalised FD tables from the Union tables
import json
import pandas as pd
def indexed(t, indexes=()):
    indexes = ['page title', 'table heading','uri'] + list(indexes)
    index_cols = [cs for cs in t.df.columns if any(c.startswith(i) for i in indexes for c in cs) ]
    return t.df.set_index(index_cols)

def make_guessed_numeric(df):
    df = df.copy()
    for col in df:
        numcol = col.replace('[^\d\.]', '', regex=True)
        df[col] = pd.to_numeric(numcol, errors='coerce')
    return df

def split_fk(df, fk):
    cat = df[(fk,)].astype('category').cat
    fk_df = pd.DataFrame(cat.categories).reset_index()
    fk_df['index'] = fk_df['index'].astype('str')
    fk_df.columns = [('PK',), ('rdf-schema#label',)]
    df = df.drop(columns=(fk,))
    df['FK'] = cat.codes.astype('str')
    return df, fk_df

def decompose_fd_tables(df, keys):
    for c in df.columns:
        if (c not in keys) and (len(set(df[c])) > 1):
            fd_df = df[[c] + list(keys)]
            
            # get filled unique rows
            filled_mask = fd_df.fillna(False).applymap(bool).all(axis=1)
            fd_df = fd_df[filled_mask].drop_duplicates(ignore_index=True)
            
            _id = f"fd_{c[0].replace(' ','_')}"
            yield takco.Table(head=zip(*fd_df.columns), body=fd_df.values, _id=_id)

def binary_stitch(ts, key):
    df = pd.concat([indexed(t, [key]) for t in ts if key in t.head[0]]).reset_index().fillna('')
    for att in df:
        if looks_numeric(df[att]):
            df[att] = make_numeric(df[att]).fillna('').astype('str')
    yield make_guessed_numeric(df)
        
def binary_fds(dfs, key):
    fd_dfs = [] 
    for df in dfs:
        keys = [('FK',)]
        if all(k in df.columns for k in keys):
            for fdt in decompose_fd_tables(df, keys):
                yield fdt, True
        else:
            _id = '_'.join('-'.join(c) for c in df.columns)
            fdt = takco.Table(head=zip(*df.columns), body=df.values, _id=_id)
            yield fdt, False

def write_snow(t, name, fd_path):
    doc = takco.evaluate.dataset.WebDataCommons.convert_back(t, snow=True)
    fname = Path(fd_path).joinpath(name)
    with open(fname, 'w') as fw:
        json.dump(doc, fw, ensure_ascii=False)


dataset_name = 'flightaware.com'
fd_path = ROOT.joinpath(f'{dataset_name}/normalised_X_fd_relations')
!rm -r $fd_path
Path(fd_path).mkdir(parents=True,exist_ok=True)

ts = list(takco.TableSet.dataset(benchmark_datasets[dataset_name][0]))
key = 'carrier'
for stitched in binary_stitch(ts, key):
    stitched = extract_bracket_disambiguation(stitched)
    print('Stitched:', stitched.shape)
    print(' ', '|'.join(c for cs in stitched.columns for c in cs) )

    prefix = f"Airline_{dataset_name}"
    n_fds = 0
    for t, is_fd in binary_fds(split_fk(stitched, key), key):
        if is_fd:
            name = f"{prefix}_fd_{n_fds}.json"
            n_fds += 1
        else:
            name = f"{prefix}.json"
        print(name)
        write_snow(t, name, fd_path)
    
fd_dataset = takco.evaluate.dataset.WebDataCommons(fnames=list(fd_path.glob("*.json")))
takco.TableSet.dataset(fd_dataset).preview(ntables=None)

Stitched: (19603, 23)
  page title|table heading|uri 0|uri 1|uri 2|uri 3|uri 4|carrier|average per flight|cargo weight (lbs)|flights performed|flights scheduled|mail transport (lbs)|maximum|median|minimum|passengers|percentage flown|percentage of seats filled|popularity|routing|total seats|disambiguation of page title
Airline_flightaware.com_fd_0.json
Airline_flightaware.com_fd_1.json
Airline_flightaware.com_fd_2.json
Airline_flightaware.com_fd_3.json
Airline_flightaware.com_fd_4.json
Airline_flightaware.com_fd_5.json
Airline_flightaware.com_fd_6.json
Airline_flightaware.com_fd_7.json
Airline_flightaware.com_fd_8.json
Airline_flightaware.com_fd_9.json
Airline_flightaware.com_fd_10.json
Airline_flightaware.com_fd_11.json
Airline_flightaware.com_fd_12.json
Airline_flightaware.com_fd_13.json
Airline_flightaware.com_fd_14.json
Airline_flightaware.com_fd_15.json
Airline_flightaware.com_fd_16.json
Airline_flightaware.com_fd_17.json
Airline_flightaware.com_fd_18.json
Airline_flightaware.com.j

?,0,1
,total seats,FK
,100.0,60
,97910.0,134
,100.0,3
,9632.0,79
,12202.0,82
?,0,1
,uri 4,FK
,JIA,134
,JIA,82


In [373]:
gold_fd_path = ROOT.joinpath('flightaware.com/normalised_S')
gold_fd_dataset = takco.evaluate.dataset.WebDataCommons(fnames=list(gold_fd_path.glob("*.json")))
takco.TableSet.dataset(gold_fd_dataset).preview(ntables=None)

?,0,1,2,3
,uri 2,uri 3,average per flight,FK
,kcmh,kjfk,41.0,Airline_flightaware.com~Row2
,kcmh,kjfk,37.0,Airline_flightaware.com~Row14
,kdfw,kdtw,111.0,Airline_flightaware.com~Row20
,kavp,kord,42.0,Airline_flightaware.com~Row1
,kdfw,kdtw,86.0,Airline_flightaware.com~Row10
?,0,1,2,3
,uri 2,uri 3,flights scheduled,FK
,krdu,ktpa,0.0,Airline_flightaware.com~Row10
,kelp,ksat,2.0,Airline_flightaware.com~Row1


## annotation analysis

In [404]:
list(fd_path.glob("*.json"))

[PosixPath('/export/scratch1/home/kruit/snow/datasets/www.vgchartz.com/normalised_X_fd_relations/VideoGame_www.vgchartz.com_fd_23.json'),
 PosixPath('/export/scratch1/home/kruit/snow/datasets/www.vgchartz.com/normalised_X_fd_relations/VideoGame_www.vgchartz.com_fd_25.json'),
 PosixPath('/export/scratch1/home/kruit/snow/datasets/www.vgchartz.com/normalised_X_fd_relations/VideoGame_www.vgchartz.com_fd_2.json'),
 PosixPath('/export/scratch1/home/kruit/snow/datasets/www.vgchartz.com/normalised_X_fd_relations/VideoGame_www.vgchartz.com_fd_20.json'),
 PosixPath('/export/scratch1/home/kruit/snow/datasets/www.vgchartz.com/normalised_X_fd_relations/VideoGame_www.vgchartz.com_fd_19.json'),
 PosixPath('/export/scratch1/home/kruit/snow/datasets/www.vgchartz.com/normalised_X_fd_relations/VideoGame_www.vgchartz.com_fd_7.json'),
 PosixPath('/export/scratch1/home/kruit/snow/datasets/www.vgchartz.com/normalised_X_fd_relations/VideoGame_www.vgchartz.com_fd_3.json'),
 PosixPath('/export/scratch1/home/kru

In [406]:
fd_dataset = takco.evaluate.dataset.WebDataCommons(fnames=list(fd_path.glob("*.json")))
takco.TableSet.dataset(fd_dataset).preview(ntables=None)

AttributeError: 'float' object has no attribute 'replace'

,0,1
,PK,rdf-schema#label
0,VideoGame_www.vgchartz.com~Row0,angry birds star wars
1,VideoGame_www.vgchartz.com~Row1,assassin's creed iv: black flag
2,VideoGame_www.vgchartz.com~Row2,battlefield 4
3,VideoGame_www.vgchartz.com~Row3,blacklight: retribution
4,VideoGame_www.vgchartz.com~Row4,call of duty: ghosts


In [58]:
# what is the difference between clusters in entity_structure and union_goldstandard?

import pandas as pd
from pathlib import Path

root = Path('~/snow/datasets/d3football.com/').expanduser().absolute()

ev = root.joinpath('evaluation/')
entity_structure = pd.read_csv(ev.joinpath('entity_structure.tsv'), sep='\t', header=None)
es = entity_structure[2].map(lambda x:tuple(set(x.split(','))))

union_goldstandard = pd.read_csv(ev.joinpath('union_goldstandard.tsv'), sep='\t', header=None)
ug = union_goldstandard[1].map(lambda x:tuple(set(x.split(','))))
ug_name = dict(zip(ug, union_goldstandard[0]))

def make_gs(root):
    fname = root.joinpath('evaluation/union_goldstandard.tsv')
    ugs_cols_df = pd.read_csv(fname, sep='\t', header=None)
    ugs_cols_df[1] = ugs_cols_df[1].map(lambda x: set(x.split(',')))
    return ugs_cols_df

gs = make_gs(root)
# for c in (set(ug) - set(es)):
#     print(c, ug_name[c])
    
# set(es) & set(ug)
gs

,0,1
0,-20: [-20],{7.json~Col12}
1,all: [all],{0.json~Col8}
2,att: [att],{4.json~Col7}
3,avg: [avg],"{7.json~Col9, 1.json~Col9, 9.json~Col9, 4.json..."
4,c-a: [c-a],{2.json~Col7}
5,collin janssen jr (baseball): [collin janssen ...,{10.json~Col7}
6,conf: [conf],{0.json~Col7}
7,email: [email],{11.json~Col7}
8,fg: [fg],{8.json~Col7}
9,int: [int],{2.json~Col10}


In [437]:
%%time
def read_eval_scores(fname):
    with open(fname) as fo:
        lines = fo.readlines()
        try:
            ind = lines.index('Overall Performance:\n')
            scores = [float(l.split(' ')[1]) for l in lines[ind+1:ind+4]]
        except:
            scores = (0,0,0)
        return dict(zip('prf', scores))
    
import subprocess
cmd = 'bash', 'evaluate_containment_specific', f'datasets/{dataset_name}', 'X'
eval_out = subprocess.run(cmd, cwd=ROOT.parent, check=True, capture_output=True)
print(eval_out.stderr.decode())
print(read_eval_scores(ROOT.joinpath(f'{dataset_name}/evaluate_containment_X.log')))

/export/scratch1/home/kruit/snow/target/snow-1.0.jar
de.uni_mannheim.informatik.dws.tnt.match.cli.EvaluateEntityStitchedUnionTables version 2021-02-15 22:36:40
		 __      __.___        __                     
		 /  \    /  \   | _____/  |_  ____     _______ 
		 \   \/\/   /   |/    \   __\/ __ \    \_  __ \
		  \        /|   |   |  \  | \  ___/     |  | \/
		   \__/\  / |___|___|  /__|  \___  > /\ |__|   
		        \/           \/          \/  \/        

{'p': 0.727268, 'r': 0.289974, 'f': 0.414628}


In [57]:
# if we cluster on attribute name only, is that similar to some annotations?

from takco.evaluate.dataset import WebDataCommons
import takco
from pathlib import Path

# root = Path('~/snow/datasets/data.bls.gov/').expanduser().absolute()
root = Path('~/snow/datasets/d3football.com/').expanduser().absolute()

fnames = root.joinpath('union_dedup_json/')
inp = WebDataCommons(fnames=list(fnames.glob("*.json")))
tableset = takco.TableSet.dataset(inp)

# Get columns per header value
head_cols = {}
for t in tableset.tables:
    fname = Path(t['fname']).name
    hs = [c.get('text') for cs in t['tableHeaders'] for c in cs]
    for hi, h in enumerate(hs):
        cid = f"{fname}~Col{hi}"
        head_cols.setdefault(h, set()).add(cid)

trivial = set(map(tuple, head_cols.values()))
print(len(set(es) & set(ug)), 'gold and entity_struct', )
print(len(set(trivial) & set(ug)), 'trivial and gold')
print(len(set(es) & set(trivial)), 'trivial and entity_struct')

20 gold and entity_struct
20 trivial and gold
16 trivial and entity_struct
